In [1]:
import torch
from torch import nn, optim
import numpy as np
import pandas as pd
import collections
import statistics
import time
import csv
import os
import sys
import argparse
import psutil
from river import base, stats, utils, drift, metrics, preprocessing, datasets
from river.datasets import synth
from deep_river import classification

In [2]:
# --- 1. FUNÇÕES DE SUPORTE ---
def generate_rotation_matrix(n_features):
    random_matrix = torch.randn(n_features, n_features)
    q, _ = torch.linalg.qr(random_matrix)
    return q

def log_results_to_csv(filename, data_dict):
    file_exists = os.path.isfile(filename)
    with open(filename, mode='a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=data_dict.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(data_dict)

In [3]:
# --- 2. CONFIGURAÇÃO DE DATASETS  ---
def get_dataset(name, seed, n_classes=2):
    """Retorna o stream e as dimensões esperadas pós-encoding."""
    if name.lower() == 'elec2':
        # Elec2 tem 8 features originais -> ~17 após OneHot das categorias
        return datasets.Elec2(), 17, 2
    elif name.lower() == 'agrawal':
        return synth.Agrawal(seed=seed), 10, 2
    elif name.lower() == 'sea':
        return synth.SEA(seed=seed), 3, 2
    elif name.lower() == 'sine':
        return synth.Sine(seed=seed), 3, 2
    elif name.lower() == 'hyperplane':
        return synth.Hyperplane(seed=seed, n_features=10), 10, 2
    elif name.lower() == 'randomtree':
        # Usando os parâmetros específicos fornecidos
        return synth.RandomTree(
            seed_tree=seed, 
            seed_sample=seed, 
            n_classes=n_classes, 
            n_num_features=5, 
            n_cat_features=5
        ), 15, n_classes # 5 num + 5 cat (que expandem no OHE)
    else:
        raise ValueError(f"Dataset {name} não configurado.")

In [4]:
# --- 3. CLASSE DAS REDES NEURAIS (Mix Heterogêneo) ---
class FlexibleNeuralNetwork(nn.Module):
    def __init__(self, n_features, n_classes, hidden_layers=[32], use_cnn=False, projection_matrix=None):
        super().__init__()
        self.n_features = n_features
        self.use_cnn = use_cnn
        if projection_matrix is not None:
            self.register_buffer('projection', projection_matrix.to(torch.float32))
        else:
            self.projection = None
        self.in_dim = (8 * n_features) if use_cnn else n_features
        if use_cnn:
            self.cnn_block = nn.Sequential(nn.Conv1d(1, 8, 3, padding=1), nn.ReLU(), nn.Flatten())
        else:
            self.cnn_block = None
        layers = []
        curr = self.in_dim
        for h in hidden_layers:
            layers.append(nn.Linear(curr, h))
            layers.append(nn.ReLU())
            curr = h
        layers.append(nn.Linear(curr, n_classes))
        self.mlp_head = nn.Sequential(*layers)

    def forward(self, x):
        if x.dim() == 1: x = x.unsqueeze(0)
        x = x.to(torch.float32)
        if x.shape[1] != self.n_features:
            tmp = torch.zeros((x.shape[0], self.n_features), device=x.device)
            tmp[:, :min(x.shape[1], self.n_features)] = x[:, :min(x.shape[1], self.n_features)]
            x = tmp
        if self.projection is not None: x = torch.matmul(x, self.projection)
        if self.use_cnn: x = self.cnn_block(x.unsqueeze(1))
        return self.mlp_head(x)

In [5]:
class FlexibleNN(nn.Module):
    def __init__(self, n_features, n_classes, hidden_layers=[10]):
        super(FlexibleNN, self).__init__()
        layers = []
        in_dim = n_features
        
        for h_dim in hidden_layers:
            layers.append(nn.Linear(in_dim, h_dim))
            layers.append(nn.ReLU())
            in_dim = h_dim
            
        layers.append(nn.Linear(in_dim, n_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [6]:
class ARTELight(base.Ensemble, base.Classifier):
    def __init__(self, models, drift_detector, lambda_val=10.0, seed=42):
        super().__init__(models=models)
        self.lambda_val = lambda_val
        self._rng = np.random.RandomState(seed)
        self.drift_detector = drift_detector
        self._detectors = [drift_detector.clone() for _ in range(len(models))]
        self._acc_windows = [utils.Rolling(stats.Mean(), window_size=100) for _ in range(len(models))]
        self._total_drifts = 0

    def learn_one(self, x, y):
        any_drift = False # Variável para controlar se houve drift neste passo
        for i, model in enumerate(self.models):
            y_pred = model.predict_one(x)
            correct = (y == y_pred)
            
            self._detectors[i].update(0 if correct else 1)
            self._acc_windows[i].update(1 if correct else 0)
            
            if not correct:
                k = self._rng.poisson(self.lambda_val)
                for _ in range(k):
                    model.learn_one(x, y)

            if self._detectors[i].drift_detected:
                self._total_drifts += 1
                self.models[i] = model.clone()
                self._detectors[i] = self.drift_detector.clone()
                self._acc_windows[i] = utils.Rolling(stats.Mean(), window_size=100)
                any_drift = True # Sinaliza que houve ao menos um drift
                
        return any_drift # Retorna para o main() decidir sobre o gc.collect()

    def predict_proba_one(self, x):
        accs = [w.get() for w in self._acc_windows]
        avg = sum(accs)/len(accs) if accs else 0
        idx = [i for i, a in enumerate(accs) if a >= avg] or range(len(self.models))
        
        votes = collections.Counter()
        for i in idx:
            p = self.models[i].predict_proba_one(x)
            for c, v in p.items(): 
                votes[c] += v / len(idx)
        return votes

    @property
    def total_drifts(self): 
        return self._total_drifts

In [7]:
# --- WRAPPER PARA DESBALANCEAMENTO (Replicando BinaryImbalancedGenerator.java) ---
class ImbalancedStream:
    def __init__(self, stream, ir=0.1, seed=42):
        self.stream = stream
        self.ir = ir # Ex: 0.05 para 5% de classe minoritária
        self.rng = np.random.RandomState(seed)
        self.iterator = iter(self.stream)

    def __iter__(self):
        return self

    def __next__(self):
        # Decide qual classe queremos com base na probabilidade IR
        target_class = 1 if self.rng.rand() < self.ir else 0
        
        # Consome o stream original até achar a classe desejada
        while True:
            try:
                x, y = next(self.iterator)
                y_val = int(y) if isinstance(y, (int, float, bool, np.bool_)) else int(y)
                if y_val == target_class:
                    return x, y_val
            except StopIteration:
                raise StopIteration

In [10]:
# --- 4. EXECUÇÃO DO EXPERIMENTO ---
def main(dataset='elec2', ir=None, seed=123456789, exp_type='binary', n_models=30, n_classes=2, batch_size=32):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 1. Obter Stream e Configuração (respeitando n_classes para sintéticos)
    stream_raw, n_feat, ds_classes = get_dataset(dataset, seed, n_classes)
    # Se o dataset real tiver número fixo de classes (como Elec2), usamos o do dataset
    final_n_classes = ds_classes if dataset.lower() != 'randomtree' else n_classes
    
    # 2. Aplicar Wrapper de Desbalanceamento (Protocolo Alberto Cano)
    # Se ir=None, o stream segue original (ex: Elec2)
    if ir is not None:
        print(f"Aplicando Static Imbalance Ratio: {ir*100}%")
        stream = ImbalancedStream(stream_raw, ir=ir, seed=seed)
    else:
        stream = stream_raw

    loss_f = nn.CrossEntropyLoss() # Definida uma única vez para todos
    # Mix de Arquiteturas Heterogêneas
    ensemble_list = []
    torch.manual_seed(seed)
    for i in range(n_models):
        if i < n_models//3:
            cfg = {"cnn": False, "opt": optim.SGD, "lr": 0.05, "layers": [128, 64], "proj": False}
        elif i < 2*n_models//3:
            cfg = {"cnn": True, "opt": optim.Adam, "lr": 0.01, "layers": [64], "proj": False}
        else:
            cfg = {"cnn": False, "opt": optim.Adam, "lr": 0.005, "layers": [256, 128], "proj": True}
        
        proj = torch.randn(n_feat, n_feat) if cfg["proj"] else None
        if proj is not None: proj, _ = torch.linalg.qr(proj)

        m = classification.Classifier(
            module=FlexibleNeuralNetwork(n_feat, final_n_classes, cfg["layers"], cfg["cnn"], proj),
            loss_fn=loss_f, 
            optimizer_fn=cfg["opt"], 
            lr=cfg["lr"], 
            device=device, 
            is_feature_incremental=False
        )
        ensemble_list.append(m)

    model_artelmb = ARTELight(ensemble_list, drift.ADWIN(delta=1e-3), lambda_val=10.0, seed=seed)
    # model_artelmb = ARTELightMB(ensemble_list, drift.ADWIN(delta=0.002), lambda_val=1.0, batch_limit=32, seed=seed)
    
    # Métricas do Survey [cite: 415, 469]
    metric_acc = metrics.Accuracy()
    metric_kappa = metrics.CohenKappa()
    metric_gmean = metrics.GeometricMean()
    
    oh_encoder = preprocessing.OneHotEncoder()
    scaler = preprocessing.StandardScaler()
    X_batch, y_batch, latencies = [], [], []
    # Organização de arquivos seguindo o padrão do repositório
    os.makedirs(f"results/{exp_type}", exist_ok=True)
    ir_str = f"_ir{ir}" if ir else ""
    output_file = f"results/{exp_type}/{dataset}{ir_str}_s{seed}_m{n_models}.csv"
    
    print(f"Executando: {dataset.upper()} | Tipo: {exp_type} | Seed: {seed} | N_Feat: {n_feat}")

    for count, (x, y) in enumerate(stream, 1):
        # Conversão de label (funciona para binário e multiclasse)
        y_val = int(y) if isinstance(y, (int, float, bool)) else y
        X_batch.append(x); y_batch.append(1 if y else 0)
        
        if len(X_batch) == batch_size:
            start = time.perf_counter()

            # Preprocessing
            X_df = pd.DataFrame(X_batch)
            oh_encoder.learn_many(X_df)
            X_oh = oh_encoder.transform_many(X_df).astype(np.float64)
            if hasattr(X_oh, "sparse"): X_oh = X_oh.sparse.to_dense()
            scaler.learn_many(X_oh)
            X_sc = scaler.transform_many(X_oh)
            
            # Padding dinâmico baseado no n_feat da configuração
            data_fix = np.zeros((batch_size, n_feat))
            cols = min(X_sc.shape[1], n_feat)
            data_fix[:, :cols] = X_sc.iloc[:, :cols].values
            X_dicts = pd.DataFrame(data_fix, columns=[f"f{j}" for j in range(n_feat)]).to_dict(orient='records')

            # Test-then-Train
            any_drift = False
            for i in range(batch_size):
                y_pred = model_artelmb.predict_one(X_dicts[i])
                if y_pred is not None:
                    metric_acc.update(y_batch[i], y_pred)
                    metric_kappa.update(y_batch[i], y_pred)
                    metric_gmean.update(y_batch[i], y_pred)
                # O learn_one agora retorna True se houver drift
                if model_artelmb.learn_one(X_dicts[i], y_batch[i]):
                    any_drift = True

            # Limpeza de memória fora do loop de instâncias, apenas se necessário
            if any_drift:
                import gc
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

            latencies.append((time.perf_counter() - start) * 1000)
            X_batch, y_batch = [], []

            if count % 2000 < batch_size:
                ram = psutil.Process().memory_info().rss / (1024 * 1024)
                vram = torch.cuda.memory_allocated() / (1024*1024) if torch.cuda.is_available() else 0
                avg_lat = sum(latencies[-20:])/20 / batch_size
                
                stats_dict = {
                    "Instancia": count, 
                    "Dataset": dataset, 
                    "Seed": seed,
                    "Accuracy": metric_acc.get(), 
                    "Kappa": metric_kappa.get(),
                    "GMean": metric_gmean.get(),
                    "Latencia_ms": avg_lat, 
                    "Drifts": model_artelmb.total_drifts,
                    "RAM_MB": ram, 
                    "VRAM_MB": vram
                }
                log_results_to_csv(output_file, stats_dict)
                print(f"Inst: {count} | Acc: {stats_dict['Accuracy']:.2%} | Kappa: {stats_dict['Kappa']:.2f} | Drifts: {stats_dict['Drifts']}")

In [11]:
if __name__ == "__main__":
    # Verifica se estamos no Jupyter ou Terminal
    # O Jupyter geralmente tem 'ipykernel_launcher' ou '-f' nos argumentos
    is_jupyter = any('ipykernel' in arg or '-f' in arg for arg in sys.argv)
    
    # Se rodado via linha de comando
    if not is_jupyter and len(sys.argv) > 1:
        parser = argparse.ArgumentParser()
        parser.add_argument('--dataset', type=str, default='elec2')
        parser.add_argument('--seed', type=int, default=123456789)
        parser.add_argument('--batch_size', type=int, default=32)
        parser.add_argument('--exp_type', type=str, default='real_world')
        parser.add_argument('--n_models', type=int, default=30)
        parser.add_argument('--n_classes', type=int, default=2)
        
        args = parser.parse_args()
        
        main(dataset=args.dataset, seed=args.seed, batch_size=args.batch_size, 
             n_models=args.n_models, n_classes=args.n_classes)
    else:
        # Execução via Notebook (parâmetros manuais abaixo)
        print("Ambiente Notebook detectado. Usando parâmetros padrão...")
        # main(dataset='agrawal', ir=0.1, seed=123456789, exp_type='static_ir')
        # main(dataset='agrawal', ir=0.01, seed=123456789, exp_type='static_ir')
        main(
            dataset='elec2', 
            seed=123456789, 
            exp_type='binary', 
            batch_size=32,
            n_models=30, n_classes=2
        )
        
        # main(dataset='elec2', seed=123456789, exp_type='binary', batch_size=32, n_models=30, n_classes=2)


Ambiente Notebook detectado. Usando parâmetros padrão...
Executando: ELEC2 | Tipo: binary | Seed: 123456789 | N_Feat: 17
Inst: 2016 | Acc: 79.81% | Kappa: 0.58 | Drifts: 58
Inst: 4000 | Acc: 78.67% | Kappa: 0.55 | Drifts: 71
Inst: 6016 | Acc: 78.56% | Kappa: 0.55 | Drifts: 73
Inst: 8000 | Acc: 78.11% | Kappa: 0.55 | Drifts: 99
Inst: 10016 | Acc: 78.36% | Kappa: 0.56 | Drifts: 112
Inst: 12000 | Acc: 78.55% | Kappa: 0.56 | Drifts: 122
Inst: 14016 | Acc: 79.11% | Kappa: 0.58 | Drifts: 150
Inst: 16000 | Acc: 79.25% | Kappa: 0.58 | Drifts: 165
Inst: 18016 | Acc: 79.42% | Kappa: 0.58 | Drifts: 172
Inst: 20000 | Acc: 79.88% | Kappa: 0.59 | Drifts: 179
Inst: 22016 | Acc: 79.91% | Kappa: 0.59 | Drifts: 182
Inst: 24000 | Acc: 80.00% | Kappa: 0.59 | Drifts: 190
Inst: 26016 | Acc: 80.35% | Kappa: 0.60 | Drifts: 202
Inst: 28000 | Acc: 80.62% | Kappa: 0.60 | Drifts: 237
Inst: 30016 | Acc: 80.89% | Kappa: 0.61 | Drifts: 247
Inst: 32000 | Acc: 80.92% | Kappa: 0.61 | Drifts: 265
Inst: 34016 | Acc: 80.9